##  Study on how to give initial data. 

We want to have a very isotropic data with a given frequency, or perhaps a distribution (thermal?) for $phi$ and its time derivative. Here
We do in 2dim.

We first create a distribution of values on Fourier space both for $ϕ $ and its time derivative, $\phi_t$.
I think we need a thermal distribution, that is we fix a temperature and put particles at that temperature.
That is we shall use the $\omega = |k|$ rule, we take positive so as to have particles and not antiparticles.



In [1]:
using Plots
using Distributions
using FFTW
using LinearAlgebra
using Revise
using Base.Threads
using HDF5

In [2]:
includet("../PIC/PIC-1D/aux_functions/inidat_aux_functions.jl")
includet("../PIC/PIC-1D/aux_functions/aux_functions.jl")
includet("local_aux_functions.jl")

We first look at the scheme for a distribution which resembles a delta function $k$ in Fourier space.
It gives a rather homogeneous distribution is space.

In [3]:
#J = (501,501,501)
J = (440,440,440)
Box_x = [-220.0, 220.0, -220.0, 220.0, -220.0, 220.0]
dx = differentials(Box_x,J)
κ = 2π./(Box_x[2:2:end] - Box_x[1:2:end-1])
V = volume(Box_x)
ϕ = Array{ComplexF64}(undef,J...)
ϕ_t = copy(ϕ)
#g = copy(ϕ)
#g_t = copy(ϕ)
#g_p = copy(ϕ)

m = 1
m2 = m^2

N = 1000 #number of particles
 
# k0= 500; δ = 1000 # para J=2001
k0 = 1; δ = 0.5


0.5

In [4]:
dx

3-element Vector{Float64}:
 1.0
 1.0
 1.0

In [5]:
k0 = 1.2; δ = 0.5 #coarse k0=0.5
h_pars = k0
f_pars = (k0, δ)
#g, g_p = get_fourier_data(f,Box_x,J,1,f_pars)
g, g_t, g_p = get_fourier_data(h,Box_x,J,1,h_pars)

(ComplexF64[-0.3437101406845707 + 0.6905209896965536im 0.35640549370295205 - 0.6840105046060102im … -0.13812741020694563 + 0.7587056221834224im -0.5819094125149531 + 0.5062377721777741im; -0.3387108685907145 - 0.6929431389742867im -0.054482983515402414 + 0.7693284199466797im … -0.7711149939362868 + 0.005872918049120207im 0.768955370772682 + 0.05951681266870285im; … ; -0.6548555409587816 + 0.40728076686579906im -0.45520958846138626 + 0.6224444196975233im … 0.7650510976949515 + 0.09574978502678952im 0.16565107136920074 + 0.7531351456372269im; 0.7327970527053634 + 0.2406319303403991im 0.6990892747640061 - 0.3257434560207316im … -0.7475916794173357 - 0.18910184003795075im -0.30135928685753594 - 0.7099416831367994im;;; -0.6962201154694498 + 0.3319228783448343im -0.08554672811562826 + 0.7664961647256332im … -0.4956745967456205 + 0.5907279569995293im -0.179454426043926 - 0.7500871430731495im; 0.4711992319447547 - 0.6105783299831953im -0.5423711011834947 + 0.5482769256904862im … -0.34101672656

In [6]:

@show sum(abs.(g))*prod(κ)
kx, ky, kz = get_frequencies(Box_x,J)
#kx_p = sort(kx)
#ky_p = sort(ky)
#kz_p = sort(kz)
#heatmap(ky_p,kx_p,real.(g_p[:,:,J[3]÷2]), aspectratio = 1)
#surface(ky_p,kx_p,real.(g[:,:]), aspectratio = 1)


sum(abs.(g)) * prod(κ) = 4.812550685855456


([0.0, 0.014279966607226332, 0.028559933214452663, 0.042839899821678996, 0.057119866428905326, 0.07139983303613166, 0.08567979964335799, 0.09995976625058432, 0.11423973285781065, 0.128519699465037  …  -0.14279966607226333, -0.128519699465037, -0.11423973285781065, -0.09995976625058432, -0.08567979964335799, -0.07139983303613166, -0.057119866428905326, -0.042839899821678996, -0.028559933214452663, -0.014279966607226332], [0.0, 0.014279966607226332, 0.028559933214452663, 0.042839899821678996, 0.057119866428905326, 0.07139983303613166, 0.08567979964335799, 0.09995976625058432, 0.11423973285781065, 0.128519699465037  …  -0.14279966607226333, -0.128519699465037, -0.11423973285781065, -0.09995976625058432, -0.08567979964335799, -0.07139983303613166, -0.057119866428905326, -0.042839899821678996, -0.028559933214452663, -0.014279966607226332], [0.0, 0.014279966607226332, 0.028559933214452663, 0.042839899821678996, 0.057119866428905326, 0.07139983303613166, 0.08567979964335799, 0.099959766250584

In [7]:
#ifft(g,J...)
p_i = plan_ifft(g; flags=FFTW.ESTIMATE, timelimit=Inf);

In [8]:
ϕ = p_i*g*sqrt(prod(J));
ϕ_t = p_i*g_t*sqrt(prod(J));


In [9]:

sum(ϕ.*conj.(ϕ))/prod(J)

0.012938414659767918 + 0.0im

In [10]:

@show norm(ϕ)
@show mean(ϕ)
@show var(ϕ);
@show n(ϕ,ϕ_t,Box_x,J);

norm(ϕ) = 1049.8313742585854
mean(ϕ) = -3.724031371444064e-5 + 7.481658304142102e-5im
var(ϕ) = 0.012938407827293623
n(ϕ, ϕ_t, Box_x, J) = (1.802814846292287e6, 0.021163773082882784)


In [11]:
p = 4 #polinomial order
percent = 0.550 #0.6 goes up to the border in poli_cut use 0.550 for step_cut
δ = 0.3 #less than 1!
#x,y,z = get_coords(Box_x,J)
#ϕ_h_cut = polinomial_cut(ϕ,Box_x,J,p,percent);
#ϕ_h_step_cut = step_cut(ϕ,Box_x,J,p,percent,δ)
#heatmap(x,y,real.(ϕ_h_step_cut[:,:,J[3]÷2]), aspectratio=1)
#heatmap(real.(ϕ_h_cut[J[1]÷2,:,:]), aspectratio=1)
#heatmap(real.(ϕ_h_cut[:,J[2]÷2,:]), aspectratio=1)
#heatmap(imag.(ϕ[:,J[2]÷2,:]), aspectratio=1)
#surface(x,y,real.(ϕ_h_step_cut[:,:,J[3]÷2]), aspectratio=1)
#png("h_cut.png")

0.3

In [12]:
#ϕ_h_cut = polinomial_cut(ϕ,Box_x,J,8);
#ϕ_t_h_cut = polinomial_cut(ϕ_t,Box_x,J,8);

ϕ_h_cut = step_cut(ϕ,Box_x,J,p,percent,δ)
ϕ_t_h_cut = step_cut(ϕ_t,Box_x,J,p,percent,δ);


In [22]:
### Derivatives

using SummationByPartsOperators



#=
h5file = h5open("derivatives_3d_h_step_cut.h5", "w") do file
    write(file, "coord0", x)
    write(file, "coord1", y)
    write(file, "coord2", z)
    write(file, "nvars", [6])
    write(file, "var0", real.(Dϕ[1,:,:,:]))
    write(file, "var1", real.(Dϕ[2,:,:,:]))
    write(file, "var2", real.(Dϕ[3,:,:,:]))
    write(file, "var3", imag.(Dϕ[1,:,:,:]))
    write(file, "var4", imag.(Dϕ[2,:,:,:,:]))
    write(file, "var5", imag.(Dϕ[3,:,:,:,]))
end
=#



In [23]:
ρ = get_rho(ϕ_h_cut,ϕ_t_h_cut,Box_x,J);

In [ ]:
surface(x,y,ρ[:,:,J[3]÷2])

In [24]:

N_fields = 5.0 #Int32(1)

x, y, z = get_coords(Box_x,J)

# Open an HDF5 file in write mode
h5file = h5open("data_3d_h_step_cut_comp_XL.h5", "w") do file
    write(file, "coord0", x)
    write(file, "coord1", y)
    write(file, "coord2", z)
    write(file, "nvars", [N_fields])
    write(file, "var0", real.(ϕ_h_cut))
    write(file, "var1", imag.(ϕ_h_cut))
    write(file, "var2", real.(ϕ_t_h_cut))
    write(file, "var3", imag.(ϕ_t_h_cut))
    write(file, "var4", ρ)
end

In [ ]:
data = h5open("example_data_3d_h_step_cut.h5", "r")

In [ ]:
var0 = read(data["var0"])
heatmap(x,y,var0[:,:,J[3]÷2], aspectratio=1)
#heatmap(x,y,real.(ϕ_h_cut)[:,:,J[3]÷2], aspectratio=1)

In [ ]:
ddata = h5open("derivatives_3d_h_step_cut.h5", "r")

## Thermal distribution

We use a relativistic thermal distribution. I guess we are setting the mass to 1 but I don't know the temperature.



In [ ]:
θ = 1.0
m = 1.0
m2 = m^2
norm_rel = 2.0
p_max = 10*θ
Box_p =(-p_max,p_max,-p_max,p_max,-p_max,p_max)
#dp = differentials(Box_p,J)
L = zeros(J...)
L = Box_p[2:2:end] .- Box_p[1:2:end]
f_p = f_p_rel 
f_p_max = f_p_rel_max
par_f_p_1 = (θ,norm_rel,2)
#norm_rel = int_mid_point_f(f_p, par_f_p_1, [20_000,20_000], Box_p)
par_f_p = (θ, norm_rel,2)
interval_p = Box_p


In [ ]:
g_th = fill(0.0+im*0.0,J...)
gt_th = fill(0.0+im*0.0,J...)

N=100000
plt = Plots.scatter(labels=false)
for i in 1:N÷2
    kr = random_sampling_from_distribution_D(f_p,f_p_max,par_f_p,interval_p)
    j = get_index_p(kr,J,L)
    jm = get_index_p(-kr,J,L)
    if (abs(j[1]) > J[1] || abs(jm[1])) > J[1] || (abs(j[2]) > J[2] || abs(jm[2])) > J[2]
        println("j = $j, jm = $jm, kr = $kr")
    end
    #Plots.scatter!(plt,[j[1],jm[1]],[j[2],jm[2]],label=false)
    factor = 1.0/sqrt(2*ω([kr[1],kr[2],kr[3]],m2))
    phase_p = exp(im*rand()*2*π)
    phase_m = exp(im*rand()*2*π)
    g_th[j...] += phase_p*factor
    g_th[jm...] += phase_m*factor
    gt_th[j...] += im*phase_p/factor/2.0
    gt_th[jm...] += 2.0*im*phase_m/factor/2.0
end

#heatmap(real.(g_th))
surface(real.(g_th[30,:,:]))
#plt

In [32]:
ϕ_th = p_i*g_th*sqrt(prod(J));
ϕt_th = p_i*gt_th*sqrt(prod(J));

In [ ]:
#heatmap(imag.(ϕ_th))
surface(imag.(ϕ_th[30,:,:]))

In [ ]:
#@show sum(ϕ.*conj.(ϕ))*prod(dx)/V

n(ϕ_th,ϕt_th,Box_x,J)

### Localization



In [ ]:
#ϕ_th_cut = polinomial_cut(ϕ_th,Box_x,J,8);
#ϕ_th_step_cut = poli_step
p = 4 #polinomial order
percent = 0.550 #0.6 goes up to the border in poli_cut use 0.550 for step_cut
δ = 0.3 #less than 1!
#x,y,z = get_coords(Box_x,J)

ϕ_th_step_cut = step_cut(ϕ_th,Box_x,J,p,percent,δ)
ϕ_t_th_step_cut = step_cut(ϕt_th,Box_x,J,p,percent,δ)

In [ ]:
surface(real.(ϕ_th_step_cut[101,:,:]) )

### Data transfer

In [25]:
N_fields = 4.0 #Int32(1)

x, y, z = get_coords(Box_x,J)

# Open an HDF5 file in write mode
h5file = h5open("example_data_3d_gaussian.h5", "w") do file
    write(file, "coord0", x)
    write(file, "coord1", y)
    write(file, "coord2", z)
    write(file, "nvars", [N_fields])
    write(file, "var0", real.(ϕ))
    write(file, "var1", imag.(ϕ))
    write(file, "var2", real.(ϕ_t))
    write(file, "var3", imag.(ϕ_t))
end